# Cryptocurrency trading bot.

In [ ]:
##%pip install python-binance==0.7.5
#%pip install python-binance

In [ ]:
!python bootstrap.py

In [1]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table, get_new_tickers
from cryptocurrency.conversion_ohlcv import convert_ohlcvs_from_pairs_to_assets
from cryptocurrency.ohlcvs import download_pairs
from cryptocurrency.resample import resample
from cryptocurrency.volume_conversion import add_rolling_volumes
from tqdm import tqdm
import pandas as pd
import os
import shutil

def bootstrap_loggers(client, assets, pairs=None, additional_intervals=None, upsampled_intervals=None, 
                      download_interval='1m', exchange_info=None, as_pair=False):
    log_file = 'crypto_logs/crypto_output_log_{}.txt'
    period = 2880 if download_interval == '1m' else 60
    second_period = 60 if download_interval == '1m' else None
    base_interval = download_interval + 'in' if download_interval[-1] == 'm' else download_interval
    frequency_1min = pd.tseries.frequencies.to_offset('1min')
    frequency_1d = pd.tseries.frequencies.to_offset('1d')
    frequency = pd.tseries.frequencies.to_offset(base_interval)
    pairs[base_interval] = download_pairs(client=client, assets=assets, interval=download_interval, 
                                          period=period, second_period=second_period)
    if not as_pair:
        pairs[base_interval] = convert_ohlcvs_from_pairs_to_assets(pairs[base_interval], exchange_info)
    pairs[base_interval] = add_rolling_volumes(pairs[base_interval])
    pairs[base_interval] = pairs[base_interval].loc[pairs[base_interval].dropna().first_valid_index():]
    if additional_intervals is not None:
        for additional_interval in tqdm(additional_intervals, unit=' pair'):
            pairs[additional_interval] = resample(pairs[base_interval].copy(), interval=additional_interval)
            pairs[additional_interval] = pairs[additional_interval].tail(200)
            pairs[additional_interval].to_csv(log_file.format(additional_interval))
    truncated_frequency = 60 if frequency > frequency_1min else 1500
    pairs[base_interval] = pairs[base_interval].tail(truncated_frequency)
    pairs[base_interval].to_csv(log_file.format(base_interval))
    if upsampled_intervals is not None:
        for subminute_interval in tqdm(upsampled_intervals, unit=' pair'):
            pairs[subminute_interval] = pairs[base_interval].tail(25)
            pairs[subminute_interval] = pairs[subminute_interval].resample(subminute_interval).agg('max')
            pairs[subminute_interval] = pairs[subminute_interval].fillna(method='pad').tail(60)
            pairs[subminute_interval].to_csv(log_file.format(subminute_interval))
    return pairs

as_pair = False
directory = 'crypto_logs'
if os.path.exists(directory):
    shutil.rmtree(directory)
os.mkdir(directory)
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory=directory)
exchange_info = exchange.info
offset_s = get_timezone_offset_in_seconds()
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        offset_s=offset_s, as_pair=True)
assets = get_new_tickers(conversion_table=conversion_table)
#assets = assets[:10]

pairs = bootstrap_loggers(client=client, assets=assets, pairs={}, 
                          additional_intervals=['30min'], 
                          upsampled_intervals=['5s', '15s'], 
                          download_interval='1m', 
                          exchange_info=exchange_info, 
                          as_pair=True)
pairs['1min']

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.18s/ pair]
10 named pair [00:00, 280.71 named pair/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  2.89 pair/s]
10 named pair [00:00, 354.83 named pair/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 126.17 pair/s]


symbol,AKROBUSD,BDOTDOT,BUSDVAI,DIABUSD,FRONTBUSD,MITHBNB,MULTIBUSD,NULSUSDT,OOKIBUSD,TKOBUSD,...,AKROBUSD,BDOTDOT,BUSDVAI,DIABUSD,FRONTBUSD,MITHBNB,MULTIBUSD,NULSUSDT,OOKIBUSD,TKOBUSD
pair,open,open,open,open,open,open,open,open,open,open,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-11-10 21:47:00,0.00295,0.7384,1.047,0.320,0.1887,0.000052,3.74,0.1845,0.00346,0.200,...,111892.64224,20315.117692,942.1128,385131.1136,723441.8455,222.729784,157656.06379,1.023608e+06,260540.19070,104416.4519
2022-11-10 21:48:00,0.00295,0.7384,1.047,0.320,0.1887,0.000052,3.74,0.1844,0.00346,0.200,...,111892.64224,20315.117692,942.1128,385131.1136,723441.8455,222.729784,158324.32325,1.023608e+06,260540.19070,104416.4519
2022-11-10 21:49:00,0.00295,0.7384,1.047,0.319,0.1884,0.000052,3.74,0.1844,0.00346,0.200,...,111892.64224,20315.117692,942.1128,385159.3132,723854.8191,222.729784,158533.37429,1.023608e+06,260632.09522,104416.4519
2022-11-10 21:50:00,0.00295,0.7384,1.047,0.319,0.1882,0.000052,3.73,0.1844,0.00346,0.200,...,111892.64224,20315.117692,942.1128,385159.3132,723930.0991,222.729784,158909.73380,1.024224e+06,265435.32258,104416.4519
2022-11-10 21:51:00,0.00295,0.7384,1.047,0.319,0.1882,0.000052,3.73,0.1842,0.00346,0.200,...,111892.64224,20315.117692,942.1128,385159.3132,723930.0991,222.729784,158909.73380,1.025021e+06,265435.32258,104416.4519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-11 22:42:00,0.00266,0.7367,1.047,0.297,0.1718,0.000052,3.46,0.1725,0.00313,0.193,...,799665.85372,61226.975826,540.7717,355437.7148,645657.2754,214.881810,230832.80505,9.184545e+05,246588.10662,110740.7322
2022-11-11 22:43:00,0.00266,0.7367,1.047,0.297,0.1718,0.000052,3.46,0.1724,0.00314,0.193,...,799665.85372,61226.975826,540.7717,355437.7148,645623.4534,214.881810,230744.55325,9.186698e+05,247749.15616,110740.7322
